In [17]:
import csv
import pandas as pd
import datetime as dt
from datetime import datetime
from itertools import islice
import numpy as np
from openpyxl import load_workbook
import collections
import openpyxl as op
import math

In [18]:
beginning_date = dt.datetime(2011, 1, 1)
predict_thru = dt.datetime(2018, 8, 6)
fileName = "Small Data Set.xlsx"
r = 0.147642304763359
alpha = 0.351575403514246
a = 0.327857205096383
b = 1.36694414288691

In [19]:
exportFile = 'Model construction CWM bgnbd1.xlsx'
writer = pd.ExcelWriter(exportFile, engine = 'xlsxwriter')

In [20]:
def datevalue(date1):
    temp = dt.datetime(1899, 12, 30)    # Note, not 31st Dec but 30th!
    delta = date1 - temp
    return float(delta.days) + (float(delta.seconds) / 86400)

In [21]:
def valuedate(fh):
    fh = int(fh)
    return datetime.fromordinal(datetime(1900, 1, 1).toordinal() + fh - 2)

In [22]:
beginning_date = datevalue(beginning_date)
predict_thru = datevalue(predict_thru)

In [23]:
endDate = beginning_date + .5*(predict_thru-beginning_date)
firstTimeTo = beginning_date + .31 * .5 * (predict_thru-beginning_date)
print(endDate)

41931.0


In [24]:
# read file and drop the empty column
dfraw2 = pd.read_excel(fileName, sheet_name='Raw data2')
dfraw2 = dfraw2.drop(columns = ['Unnamed: 2'])

In [25]:
#create First Gift Column and reorder
allGiftDates = dfraw2.groupby(['Constituent ID']).describe()['Gift Date']['min']
allIDs = dfraw2.groupby(['Constituent ID']).describe()
my_list = []
for index, rows in dfraw2.iterrows():
    my_list.append(allGiftDates[(rows['Constituent ID'])])
    
dfraw2['First Gift Date'] = my_list

cols = list(dfraw2.columns.values)
dfraw2 = dfraw2[cols[0:2] + [cols[-1]] + cols[2:-1]]

In [26]:
adGiftList = [1]
for index, row in islice(dfraw2.iterrows(), 1, None):
    if(dfraw2.iloc[index-1, 0] != dfraw2.iloc[index, 0]):
        adGiftList.append(1)
    elif(dfraw2.iloc[index,4]>endDate):
            adGiftList.append(0)
    else:
        if(dfraw2.iloc[index-1, 0] == dfraw2.iloc[index, 0] and dfraw2.iloc[index-1, 4] != dfraw2.iloc[index, 4]):
            adGiftList.append(adGiftList[index-1] + 1)
        else:
            adGiftList.append(adGiftList[index-1])

dfraw2['adj. num gift'] = (adGiftList)

In [27]:
keepList = []
for index, row in islice(dfraw2.iterrows(), 1, None):
    if ((dfraw2.iloc[index-1, 3]<=endDate and dfraw2.iloc[index-1,0] != dfraw2.iloc[index,0]) or (dfraw2.iloc[index-1, 3]>endDate and dfraw2.iloc[index-1, 4]<=endDate and dfraw2.iloc[index, 4]>endDate)):
        keepList.append("")
    else:
        keepList.append(False)

        
if(dfraw2.iloc[-1, 3]<=endDate):
    keepList.append("")
else:
    keepList.append(False)
        
dfraw2['KEEP'] = (keepList)

In [28]:
rptList = [""]
for index, row in islice(dfraw2.iterrows(), 1, None):
    if (dfraw2.iloc[index-1, 0] == dfraw2.iloc[index, 0] and dfraw2.iloc[index, 4]>endDate and dfraw2.iloc[index-1, 4] == dfraw2.iloc[index, 4]):
        rptList.append("blah")
    else:
        rptList.append("")
        
dfraw2['rpt gift after cali'] = (rptList)

In [29]:
dfraw2
#df.to_excel(writer, 'Raw data2', index = False)
#writer.save()

,Constituent ID,Name,First Gift Date,Last Gift Date,Gift Date,Inactive?,adj. num gift,KEEP,rpt gift after cali
0,1927780,"Angeles, Joey D.",40547.0,43158,40547,False,1,False,
1,1927780,"Angeles, Joey D.",40547.0,43158,40578,False,2,False,
2,1927780,"Angeles, Joey D.",40547.0,43158,40606,False,3,False,
3,1927780,"Angeles, Joey D.",40547.0,43158,40637,False,4,False,
4,1927780,"Angeles, Joey D.",40547.0,43158,40667,False,5,False,
...,...,...,...,...,...,...,...,...,...
5584,1929858,"Grove, Karen E.",40970.0,42031,41828,False,4,,
5585,1929858,"Grove, Karen E.",40970.0,42031,42031,False,0,False,
5586,1929859,"Leipr, Michael H.",40970.0,40970,40970,True,1,,
5587,1929860,"Foy, Charles B.",40970.0,40970,40970,True,1,,


In [30]:
#figure out Query2

In [31]:
dfraw1 = pd.read_excel(writer, 'Raw data1')
dfraw1

,Constituent ID,Name,First Gift Date,Last Gift Date,Gift Date,Gift Amount,Gift Type,Inactive?,remove $0 & pledges
0,1927780,"Angeles, Joey D.",40547,43158,40547,50.0,Cash,False,NaN
1,1927780,"Angeles, Joey D.",40547,43158,40578,50.0,Cash,False,NaN
2,1927780,"Angeles, Joey D.",40547,43158,40606,50.0,Cash,False,NaN
3,1927780,"Angeles, Joey D.",40547,43158,40637,50.0,Cash,False,NaN
4,1927780,"Angeles, Joey D.",40547,43158,40667,50.0,Cash,False,NaN
...,...,...,...,...,...,...,...,...,...
5594,1929858,"Grove, Karen E.",40970,42031,41828,600.0,Cash,False,NaN
5595,1929858,"Grove, Karen E.",40970,42031,42031,600.0,Cash,False,NaN
5596,1929859,"Leipr, Michael H.",40970,40970,40970,750.0,Cash,True,NaN
5597,1929860,"Foy, Charles B.",40970,40970,40970,1200.0,Cash,True,NaN


In [32]:
removePledge0 = []
for index, rows in dfraw1.iterrows():
    if(dfraw1.iloc[index, 5] == 0 or dfraw1.iloc[index, 6] == ("Pledge")):
        removePledge0.append(True)
    else:
        removePledge0.append("")
dfraw1['remove $0 & pledges'] = (removePledge0)
dfraw1

,Constituent ID,Name,First Gift Date,Last Gift Date,Gift Date,Gift Amount,Gift Type,Inactive?,remove $0 & pledges
0,1927780,"Angeles, Joey D.",40547,43158,40547,50.0,Cash,False,
1,1927780,"Angeles, Joey D.",40547,43158,40578,50.0,Cash,False,
2,1927780,"Angeles, Joey D.",40547,43158,40606,50.0,Cash,False,
3,1927780,"Angeles, Joey D.",40547,43158,40637,50.0,Cash,False,
4,1927780,"Angeles, Joey D.",40547,43158,40667,50.0,Cash,False,
...,...,...,...,...,...,...,...,...,...
5594,1929858,"Grove, Karen E.",40970,42031,41828,600.0,Cash,False,
5595,1929858,"Grove, Karen E.",40970,42031,42031,600.0,Cash,False,
5596,1929859,"Leipr, Michael H.",40970,40970,40970,750.0,Cash,True,
5597,1929860,"Foy, Charles B.",40970,40970,40970,1200.0,Cash,True,


In [33]:
dfdp1 = dfraw1.copy(deep=True)
pledgeor0 = dfdp1[dfdp1['remove $0 & pledges'] == True].index
dfdp1.drop(pledgeor0, inplace=True)
dfdp1 = dfdp1.reset_index(drop=True)
dfdp1

,Constituent ID,Name,First Gift Date,Last Gift Date,Gift Date,Gift Amount,Gift Type,Inactive?,remove $0 & pledges
0,1927780,"Angeles, Joey D.",40547,43158,40547,50.0,Cash,False,
1,1927780,"Angeles, Joey D.",40547,43158,40578,50.0,Cash,False,
2,1927780,"Angeles, Joey D.",40547,43158,40606,50.0,Cash,False,
3,1927780,"Angeles, Joey D.",40547,43158,40637,50.0,Cash,False,
4,1927780,"Angeles, Joey D.",40547,43158,40667,50.0,Cash,False,
...,...,...,...,...,...,...,...,...,...
5584,1929858,"Grove, Karen E.",40970,42031,41828,600.0,Cash,False,
5585,1929858,"Grove, Karen E.",40970,42031,42031,600.0,Cash,False,
5586,1929859,"Leipr, Michael H.",40970,40970,40970,750.0,Cash,True,
5587,1929860,"Foy, Charles B.",40970,40970,40970,1200.0,Cash,True,


In [34]:
dfdp2 = dfraw2.drop(columns = 'rpt gift after cali')
falseKeep = dfdp2[dfdp2['KEEP'] == False].index
dfdp2.drop(falseKeep, inplace=True)
dfdp2 = dfdp2.reset_index(drop=True)
dfdp2

,Constituent ID,Name,First Gift Date,Last Gift Date,Gift Date,Inactive?,adj. num gift,KEEP
0,1927780,"Angeles, Joey D.",40547.0,43158,41718,False,26,
1,1927778,"Ayika, Christopher",40548.0,40548,40548,True,1,
2,1927784,"Boland, Patrick T.",40550.0,40613,40613,True,7,
3,1927785,"Callahan, Patrick J.",40554.0,40554,40554,True,1,
4,1927788,"Rowell, Esperanza",40557.0,42403,41660,False,4,
...,...,...,...,...,...,...,...,...
2021,1930534,"Chmielarczyk, Allison",40969.0,40969,40969,True,1,
2022,1929858,"Grove, Karen E.",40970.0,42031,41828,False,4,
2023,1929859,"Leipr, Michael H.",40970.0,40970,40970,True,1,
2024,1929860,"Foy, Charles B.",40970.0,40970,40970,True,1,


In [35]:
dfmd = dfdp2[["Constituent ID", "Name"]]
dfmd["x (#donations)"] = dfdp2["adj. num gift"]-1
dfmd['t_x (last gift)'] = (dfdp2['Gift Date'] - dfdp2['First Gift Date'])/365

<ipython-input-35-ea6a9f9cc4ea>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfmd["x (#donations)"] = dfdp2["adj. num gift"]-1
<ipython-input-35-ea6a9f9cc4ea>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfmd['t_x (last gift)'] = (dfdp2['Gift Date'] - dfdp2['First Gift Date'])/365


In [36]:
tempDate = endDate+1
dfmd['T (total time span)'] = (tempDate - dfdp2['First Gift Date'])/365
dfmd

<ipython-input-36-0793d730c68b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfmd['T (total time span)'] = (tempDate - dfdp2['First Gift Date'])/365


,Constituent ID,Name,x (#donations),t_x (last gift),T (total time span)
0,1927780,"Angeles, Joey D.",25,3.208219,3.794521
1,1927778,"Ayika, Christopher",0,0.000000,3.791781
2,1927784,"Boland, Patrick T.",6,0.172603,3.786301
3,1927785,"Callahan, Patrick J.",0,0.000000,3.775342
4,1927788,"Rowell, Esperanza",3,3.021918,3.767123
...,...,...,...,...,...
2021,1930534,"Chmielarczyk, Allison",0,0.000000,2.638356
2022,1929858,"Grove, Karen E.",3,2.350685,2.635616
2023,1929859,"Leipr, Michael H.",0,0.000000,2.635616
2024,1929860,"Foy, Charles B.",0,0.000000,2.635616


In [37]:
df_BGNBD = dfmd.copy(deep=True)
donationNum = df_BGNBD['x (#donations)'].tolist()
lastGift = df_BGNBD['t_x (last gift)'].tolist()
T = df_BGNBD['T (total time span)'].tolist()

In [38]:
A4 = []
for index, rows in islice(df_BGNBD.iterrows(), 0, None):
    if donationNum[index]>0:
        A4.append(math.log(a)-math.log(b+donationNum[index]-1)-(r+donationNum[index])*math.log(alpha+lastGift[index]))
    else:
        A4.append(0)
df_BGNBD['ln(A_4)'] = (A4)
A_4 = df_BGNBD['ln(A_4)'].tolist()

In [39]:
A3 = []
for index, rows in islice(df_BGNBD.iterrows(), 0, None):
    A3.append(-(r+donationNum[index])*math.log(alpha+T[index]))
df_BGNBD['ln(A_3)'] = (A3)
A_3 = df_BGNBD['ln(A_3)'].tolist()

In [40]:
A2 = []
for index, rows in islice(df_BGNBD.iterrows(), 0, None):
    A2.append(math.lgamma(a+b)+math.lgamma(b+donationNum[index])-math.lgamma(b)-math.lgamma(a+b+donationNum[index]))
df_BGNBD['ln(A_2)'] = (A2)
A_2 = df_BGNBD['ln(A_2)'].tolist()

In [41]:
A1 = []
for index, rows in islice(df_BGNBD.iterrows(), 0, None):
    A1.append(math.lgamma(r+donationNum[index])-math.lgamma(r)+r*math.log(alpha))
df_BGNBD['ln(A_1)'] = (A1)
A_1 = df_BGNBD['ln(A_1)'].tolist()
df_BGNBD

,Constituent ID,Name,x (#donations),t_x (last gift),T (total time span),ln(A_4),ln(A_3),ln(A_2),ln(A_1)
0,1927780,"Angeles, Joey D.",25,3.208219,3.794521,-36.278657,-35.764151,-1.048461,53.258612
1,1927778,"Ayika, Christopher",0,0.000000,3.791781,0.000000,-0.209874,0.000000,-0.154335
2,1927784,"Boland, Patrick T.",6,0.172603,3.786301,1.004611,-8.730776,-0.619533,3.042515
3,1927785,"Callahan, Patrick J.",0,0.000000,3.775342,0.000000,-0.209288,0.000000,-0.154335
4,1927788,"Rowell, Esperanza",3,3.021918,3.767123,-6.156554,-4.455605,-0.437634,-1.165218
...,...,...,...,...,...,...,...,...,...
2021,1930534,"Chmielarczyk, Allison",0,0.000000,2.638356,0.000000,-0.161705,0.000000,-0.154335
2022,1929858,"Grove, Karen E.",3,2.350685,2.635616,-5.458218,-3.444571,-0.437634,-1.165218
2023,1929859,"Leipr, Michael H.",0,0.000000,2.635616,0.000000,-0.161570,0.000000,-0.154335
2024,1929860,"Foy, Charles B.",0,0.000000,2.635616,0.000000,-0.161570,0.000000,-0.154335


In [42]:
ln = []
for index, rows in islice(df_BGNBD.iterrows(), 0, None):
    if donationNum[index]>0:
        ln.append(A_1[index]+A_2[index]+math.log(math.exp(A_3[index])+1*math.exp(A_4[index])))
    else:
        ln.append(A_1[index]+A_2[index]+math.log(math.exp(A_3[index])+0*math.exp(A_4[index])))
df_BGNBD['ln(.)'] = (ln)

In [43]:
BGNBDcols = list(df_BGNBD.columns.values)
df_BGNBD = df_BGNBD[BGNBDcols[0:5] + [BGNBDcols[-1]] + [BGNBDcols[-2]] + [BGNBDcols[-3]] + [BGNBDcols[-4]] + [BGNBDcols[-5]]]
df_BGNBD

,Constituent ID,Name,x (#donations),t_x (last gift),T (total time span),ln(.),ln(A_1),ln(A_2),ln(A_3),ln(A_4)
0,1927780,"Angeles, Joey D.",25,3.208219,3.794521,16.914625,53.258612,-1.048461,-35.764151,-36.278657
1,1927778,"Ayika, Christopher",0,0.000000,3.791781,-0.364210,-0.154335,0.000000,-0.209874,0.000000
2,1927784,"Boland, Patrick T.",6,0.172603,3.786301,3.427652,3.042515,-0.619533,-8.730776,1.004611
3,1927785,"Callahan, Patrick J.",0,0.000000,3.775342,-0.363623,-0.154335,0.000000,-0.209288,0.000000
4,1927788,"Rowell, Esperanza",3,3.021918,3.767123,-5.890817,-1.165218,-0.437634,-4.455605,-6.156554
...,...,...,...,...,...,...,...,...,...,...
2021,1930534,"Chmielarczyk, Allison",0,0.000000,2.638356,-0.316040,-0.154335,0.000000,-0.161705,0.000000
2022,1929858,"Grove, Karen E.",3,2.350685,2.635616,-4.922112,-1.165218,-0.437634,-3.444571,-5.458218
2023,1929859,"Leipr, Michael H.",0,0.000000,2.635616,-0.315905,-0.154335,0.000000,-0.161570,0.000000
2024,1929860,"Foy, Charles B.",0,0.000000,2.635616,-0.315905,-0.154335,0.000000,-0.161570,0.000000


In [44]:
df_ext = pd.DataFrame()
tuntil = (predict_thru - beginning_date + 1)/365
print(tuntil)
n = 1/365
t = [n]
while n <= tuntil:
    n = n + (1/365)
    t.append(n)
df_ext['t'] = t
df_ext['E(X(t))'] = 0
df_ext['2F1'] = 0
df_ext

7.602739726027397


,t,E(X(t)),2F1
0,0.002740,0,0
1,0.005479,0,0
2,0.008219,0,0
3,0.010959,0,0
4,0.013699,0,0
...,...,...,...
2770,7.591781,0,0
2771,7.594521,0,0
2772,7.597260,0,0
2773,7.600000,0,0


In [45]:
z=[]
for index, rows in islice(df_ext.iterrows(), 0, None):
    z.append(df_ext.iloc[index,0] / (alpha + df_ext.iloc[index,0]))
df_ext['z']=z
df_ext['0'] = 1
df_ext

,t,E(X(t)),2F1,z,0
0,0.002740,0,0,0.007732,1
1,0.005479,0,0,0.015346,1
2,0.008219,0,0,0.022844,1
3,0.010959,0,0,0.030229,1
4,0.013699,0,0,0.037502,1
...,...,...,...,...,...
2770,7.591781,0,0,0.955740,1
2771,7.594521,0,0,0.955755,1
2772,7.597260,0,0,0.955770,1
2773,7.600000,0,0,0.955785,1


In [46]:
count = 1
ext_cols = []
extCond = True
while (extCond):
    for index, rows in islice(df_ext.iterrows(), 0, None):
        ext_cols.append(df_ext.iloc[index, count+3]*df_ext.iloc[index, 3]*(r+count-1)*(b+count-1)/((a+b-1+count-1)*count))
    if (ext_cols[-1] < 9e-11):
        extCond = False
    else:
        df_ext[str(count)] = ext_cols
        count = count + 1
        ext_cols = [];
df_ext

,t,E(X(t)),2F1,z,0,1,2,3,4,5,...,445,446,447,448,449,450,451,452,453,454
0,0.002740,0,0,0.007732,1,0.002246,0.000014,9.626067e-08,6.922743e-10,5.076178e-12,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1,0.005479,0,0,0.015346,1,0.004458,0.000055,7.524941e-07,1.074032e-08,1.563006e-10,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
2,0.008219,0,0,0.022844,1,0.006636,0.000121,2.482092e-06,5.273556e-08,1.142401e-09,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
3,0.010959,0,0,0.030229,1,0.008780,0.000213,5.751096e-06,1.616890e-07,4.634893e-09,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
4,0.013699,0,0,0.037502,1,0.010893,0.000327,1.098175e-05,3.830380e-07,1.362201e-08,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2770,7.591781,0,0,0.955740,1,0.277613,0.212631,1.817672e-01,1.615723e-01,1.464360e-01,...,1.380304e-10,1.318676e-10,1.259800e-10,1.203555e-10,1.149821e-10,1.098488e-10,1.049447e-10,1.002596e-10,9.578381e-11,9.150788e-11
2771,7.594521,0,0,0.955755,1,0.277618,0.212637,1.817759e-01,1.615826e-01,1.464476e-01,...,1.390146e-10,1.328100e-10,1.268824e-10,1.212195e-10,1.158094e-10,1.106409e-10,1.057031e-10,1.009858e-10,9.647913e-11,9.217364e-11
2772,7.597260,0,0,0.955770,1,0.277622,0.212644,1.817846e-01,1.615930e-01,1.464593e-01,...,1.400052e-10,1.337585e-10,1.277906e-10,1.220891e-10,1.166421e-10,1.114382e-10,1.064665e-10,1.017168e-10,9.717900e-11,9.284376e-11
2773,7.600000,0,0,0.955785,1,0.277627,0.212651,1.817933e-01,1.616033e-01,1.464710e-01,...,1.410021e-10,1.347131e-10,1.287046e-10,1.229643e-10,1.174801e-10,1.122406e-10,1.072349e-10,1.024525e-10,9.788346e-11,9.351828e-11


In [47]:
df_save = df_ext
df_save

,t,E(X(t)),2F1,z,0,1,2,3,4,5,...,445,446,447,448,449,450,451,452,453,454
0,0.002740,0,0,0.007732,1,0.002246,0.000014,9.626067e-08,6.922743e-10,5.076178e-12,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1,0.005479,0,0,0.015346,1,0.004458,0.000055,7.524941e-07,1.074032e-08,1.563006e-10,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
2,0.008219,0,0,0.022844,1,0.006636,0.000121,2.482092e-06,5.273556e-08,1.142401e-09,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
3,0.010959,0,0,0.030229,1,0.008780,0.000213,5.751096e-06,1.616890e-07,4.634893e-09,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
4,0.013699,0,0,0.037502,1,0.010893,0.000327,1.098175e-05,3.830380e-07,1.362201e-08,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2770,7.591781,0,0,0.955740,1,0.277613,0.212631,1.817672e-01,1.615723e-01,1.464360e-01,...,1.380304e-10,1.318676e-10,1.259800e-10,1.203555e-10,1.149821e-10,1.098488e-10,1.049447e-10,1.002596e-10,9.578381e-11,9.150788e-11
2771,7.594521,0,0,0.955755,1,0.277618,0.212637,1.817759e-01,1.615826e-01,1.464476e-01,...,1.390146e-10,1.328100e-10,1.268824e-10,1.212195e-10,1.158094e-10,1.106409e-10,1.057031e-10,1.009858e-10,9.647913e-11,9.217364e-11
2772,7.597260,0,0,0.955770,1,0.277622,0.212644,1.817846e-01,1.615930e-01,1.464593e-01,...,1.400052e-10,1.337585e-10,1.277906e-10,1.220891e-10,1.166421e-10,1.114382e-10,1.064665e-10,1.017168e-10,9.717900e-11,9.284376e-11
2773,7.600000,0,0,0.955785,1,0.277627,0.212651,1.817933e-01,1.616033e-01,1.464710e-01,...,1.410021e-10,1.347131e-10,1.287046e-10,1.229643e-10,1.174801e-10,1.122406e-10,1.072349e-10,1.024525e-10,9.788346e-11,9.351828e-11


In [70]:
df_ext = df_save
ext = []
df_ext['2F1'] = df_ext.iloc[:,4:-1].sum(axis=1)
extConst = (a+b-1)/(a-1)
print(extConst)
for index, rows in islice(df_ext.iterrows(), 0, None):
    ext.append(extConst*(1-pow((alpha/(alpha+df_ext.iloc[index,0])),r) * df_ext.iloc[index,2]))
df_ext['E(X(t))'] = ext
df_ext

-1.0337109216248093


,t,E(X(t)),2F1,z,0,1,2,3,4,5,...,445,446,447,448,449,450,451,452,453,454
0,0.002740,0.001150,1.002260,0.007732,1,0.002246,0.000014,9.626067e-08,6.922743e-10,5.076178e-12,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1,0.005479,0.002297,1.004513,0.015346,1,0.004458,0.000055,7.524941e-07,1.074032e-08,1.563006e-10,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
2,0.008219,0.003443,1.006760,0.022844,1,0.006636,0.000121,2.482092e-06,5.273556e-08,1.142401e-09,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
3,0.010959,0.004586,1.008999,0.030229,1,0.008780,0.000213,5.751096e-06,1.616890e-07,4.634893e-09,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
4,0.013699,0.005728,1.011232,0.037502,1,0.010893,0.000327,1.098175e-05,3.830380e-07,1.362201e-08,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2770,7.591781,1.805653,4.352388,0.955740,1,0.277613,0.212631,1.817672e-01,1.615723e-01,1.464360e-01,...,1.380304e-10,1.318676e-10,1.259800e-10,1.203555e-10,1.149821e-10,1.098488e-10,1.049447e-10,1.002596e-10,9.578381e-11,9.150788e-11
2771,7.594521,1.806151,4.353372,0.955755,1,0.277618,0.212637,1.817759e-01,1.615826e-01,1.464476e-01,...,1.390146e-10,1.328100e-10,1.268824e-10,1.212195e-10,1.158094e-10,1.106409e-10,1.057031e-10,1.009858e-10,9.647913e-11,9.217364e-11
2772,7.597260,1.806649,4.354357,0.955770,1,0.277622,0.212644,1.817846e-01,1.615930e-01,1.464593e-01,...,1.400052e-10,1.337585e-10,1.277906e-10,1.220891e-10,1.166421e-10,1.114382e-10,1.064665e-10,1.017168e-10,9.717900e-11,9.284376e-11
2773,7.600000,1.807146,4.355341,0.955785,1,0.277627,0.212651,1.817933e-01,1.616033e-01,1.464710e-01,...,1.410021e-10,1.347131e-10,1.287046e-10,1.229643e-10,1.174801e-10,1.122406e-10,1.072349e-10,1.024525e-10,9.788346e-11,9.351828e-11


In [69]:
print(ext)

[0.0011495415067746663, 0.0022971087890703355, 0.003442717888319279, 0.004586384603530907, 0.005728124496683432, 0.006867952897953701, 0.008005884910798997, 0.009141935416887561, 0.010276119080892338, 0.011408450355141927, 0.012538943484146465, 0.013667612508988757, 0.014794471271601106, 0.01591953341891614, 0.017042812406907224, 0.018164321504517775, 0.019284073797483866, 0.020402082192052605, 0.021518359418601585, 0.02263291803516265, 0.023745770430851045, 0.02485692882920509, 0.025966405291440002, 0.027074211719614247, 0.028180359859717646, 0.029284861304677758, 0.030387727497290736, 0.0314889697330799, 0.03258859916307904, 0.03368662679654861, 0.034783063503623673, 0.03587792001789695, 0.036971206938939336, 0.03806293473475609, 0.039153113744186785, 0.040241754179245845, 0.041328866127407736, 0.04241445955383708, 0.0434985443035678, 0.044581130103626986, 0.04566222656511487, 0.04674184318523182, 0.04781998934925941, 0.04889667433249995, 0.049971907302164824, 0.0510456973192254, 0.0

In [49]:
calibLength = (endDate-beginning_date+1)/365
df_ns = pd.DataFrame()
df_ns["Constituent ID"] = df_BGNBD["Constituent ID"]
df_ns["T (total time span)"] = df_BGNBD["T (total time span)"]

In [50]:
firstPurch = []
T = df_ns['T (total time span)'].to_list()
for index, row in islice(df_ns.iterrows(), 0, None):
    firstPurch.append(calibLength - T[index])
df_ns['time of 1st purchase'] = (firstPurch)
maxTime = df_ns['time of 1st purchase'].max()
df_ns

,Constituent ID,T (total time span),time of 1st purchase
0,1927780,3.794521,0.008219
1,1927778,3.791781,0.010959
2,1927784,3.786301,0.016438
3,1927785,3.775342,0.027397
4,1927788,3.767123,0.035616
...,...,...,...
2021,1930534,2.638356,1.164384
2022,1929858,2.635616,1.167123
2023,1929859,2.635616,1.167123
2024,1929860,2.635616,1.167123


In [51]:
weekUntil = (predict_thru - beginning_date)/7
df_sls_top = pd.DataFrame()
trialDon = []
numDon = []
i = 1/365
while (i<=maxTime):
    trialDon.append(i)
    numDon.append(df_ns.loc[(df_ns['time of 1st purchase'] >= i-.001) & (df_ns['time of 1st purchase'] < i+.001)].count()[0])
    i = i + 1/365
df_sls_top["Time of Trial Donation"] = trialDon
df_sls_top["Number of Donors"] = numDon
df_sls_top

,Time of Trial Donation,Number of Donors
0,0.002740,0
1,0.005479,0
2,0.008219,1
3,0.010959,1
4,0.013699,0
...,...,...
420,1.153425,0
421,1.156164,3
422,1.158904,0
423,1.161644,1


In [52]:
df_sls = pd.DataFrame()
df_sls['t'] = t
df_sls['Cum. Rpt.'] = ""
df_sls['E(X(t))'] = ext
df_sls

,t,Cum. Rpt.,E(X(t))
0,0.002740,,0.001150
1,0.005479,,0.002297
2,0.008219,,0.003443
3,0.010959,,0.004586
4,0.013699,,0.005728
...,...,...,...
2770,7.591781,,1.805653
2771,7.594521,,1.806151
2772,7.597260,,1.806649
2773,7.600000,,1.807146


In [53]:
slsCount = 0
slsCols = []
while (slsCount < df_sls_top.count()[0]):
    for index, rows in islice(df_sls.iterrows(), 0, None):
        if (df_sls.iloc[index, 0] <= df_sls_top["Time of Trial Donation"][slsCount]):
            slsCols.append(0)
        else:
            indx = 365*(df_sls.iloc[index, 0]-df_sls_top["Time of Trial Donation"][slsCount])
            slsCols.append(ext[round(indx)-1])
    df_sls[str(slsCount)] = slsCols
    slsCount = slsCount+1
    slsCols = []
df_sls

,t,Cum. Rpt.,E(X(t)),0,1,2,3,4,5,6,...,415,416,417,418,419,420,421,422,423,424
0,0.002740,,0.001150,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.005479,,0.002297,0.001150,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.008219,,0.003443,0.002297,0.001150,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.010959,,0.004586,0.003443,0.002297,0.001150,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.013699,,0.005728,0.004586,0.003443,0.002297,0.001150,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2770,7.591781,,1.805653,1.805156,1.804658,1.804160,1.803662,1.803164,1.802666,1.802168,...,1.593787,1.593265,1.592743,1.592222,1.591700,1.591178,1.590656,1.590134,1.589612,1.589090
2771,7.594521,,1.806151,1.805653,1.805156,1.804658,1.804160,1.803662,1.803164,1.802666,...,1.594308,1.593787,1.593265,1.592743,1.592222,1.591700,1.591178,1.590656,1.590134,1.589612
2772,7.597260,,1.806649,1.806151,1.805653,1.805156,1.804658,1.804160,1.803662,1.803164,...,1.594830,1.594308,1.593787,1.593265,1.592743,1.592222,1.591700,1.591178,1.590656,1.590134
2773,7.600000,,1.807146,1.806649,1.806151,1.805653,1.805156,1.804658,1.804160,1.803662,...,1.595351,1.594830,1.594308,1.593787,1.593265,1.592743,1.592222,1.591700,1.591178,1.590656


In [54]:
df_save1 = df_sls

In [55]:
# cumRpt = []
# cumColCnt = 0
# #for c in range(df_sls_top.count()[0]):
# print(df_sls.iloc[0, 3:-1])
# print(df_sls_top["Time of Trial Donation"])
# print(df_sls.iloc[0, 3:-1] * df_sls_top["Time of Trial Donation"])
# #for r in range (df_sls.count()[0]):
#     #print((df_sls.iloc[r, 3:-1] * df_sls_top["Time of Trial Donation"]))
#     #cumRpt.append(sum(df_sls.iloc[r, 3:-1] * df_sls_top["Time of Trial Donation"]))
# df_sls["Cum. Rpt"] = cumRpt
# df_sls
cumRpt = []
cumRptSum = 0
#print(df_sls_top["Number of Donors"])
#print(df_sls.iloc[1,3:-1].astype('float64'))
#print (df_sls_top["Number of Donors"].multiply((df_sls.iloc[0,3:-1].astype('float64')), level = 0))
for index, rows in islice(df_sls.iterrows(), 0, None):
    i = 0
    cumRptSum = 0
    #print(cumRptSum + df_sls.iloc[index, i+3])
    #print(numDon[i])
    while (i<df_sls_top.count()[0]): 
        cumRptSum = cumRptSum + df_sls.iloc[index, i+3] * numDon[i]
        i = i+1
    cumRpt.append(cumRptSum)
    #print(cumRptSum)
df_sls["Cum. Rpt."] = cumRpt
df_sls

,t,Cum. Rpt.,E(X(t)),0,1,2,3,4,5,6,...,415,416,417,418,419,420,421,422,423,424
0,0.002740,0.000000,0.001150,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.005479,0.000000,0.002297,0.001150,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.008219,0.000000,0.003443,0.002297,0.001150,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.010959,0.001150,0.004586,0.003443,0.002297,0.001150,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.013699,0.003447,0.005728,0.004586,0.003443,0.002297,0.001150,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2770,7.591781,3450.057178,1.805653,1.805156,1.804658,1.804160,1.803662,1.803164,1.802666,1.802168,...,1.593787,1.593265,1.592743,1.592222,1.591700,1.591178,1.590656,1.590134,1.589612,1.589090
2771,7.594521,3451.085485,1.806151,1.805653,1.805156,1.804658,1.804160,1.803662,1.803164,1.802666,...,1.594308,1.593787,1.593265,1.592743,1.592222,1.591700,1.591178,1.590656,1.590134,1.589612
2772,7.597260,3452.113677,1.806649,1.806151,1.805653,1.805156,1.804658,1.804160,1.803662,1.803164,...,1.594830,1.594308,1.593787,1.593265,1.592743,1.592222,1.591700,1.591178,1.590656,1.590134
2773,7.600000,3453.141754,1.807146,1.806649,1.806151,1.805653,1.805156,1.804658,1.804160,1.803662,...,1.595351,1.594830,1.594308,1.593787,1.593265,1.592743,1.592222,1.591700,1.591178,1.590656


In [56]:
df_checkCumRpt = pd.DataFrame()
week = list(range(1, int(weekUntil)+1))
df_checkCumRpt["week"] = week
df_checkCumRpt

,week
0,1
1,2
2,3
3,4
4,5
...,...
391,392
392,393
393,394
394,395


In [57]:
checkDon = [cumRpt[6]]
checkWeekDon = [checkDon[0]]
weekCount = 2
while (weekCount <= weekUntil):
    checkDon.append(cumRpt[7*weekCount-1])
    checkWeekDon.append(checkDon[weekCount-1] - checkDon[weekCount-2])
    weekCount = weekCount + 1
df_checkCumRpt["Cum. Rpt. Dons."] = checkDon
df_checkCumRpt["Weekly Rpt. Dons."] = checkWeekDon
df_checkCumRpt

,week,Cum. Rpt. Dons.,Weekly Rpt. Dons.
0,1,0.009179,0.009179
1,2,0.039975,0.030796
2,3,0.096740,0.056765
3,4,0.202092,0.105352
4,5,0.360013,0.157921
...,...,...,...
391,392,3422.249158,7.223329
392,393,3429.466770,7.217611
393,394,3436.678682,7.211913
394,395,3443.884915,7.206232


In [58]:
startDate = [beginning_date]
endDate = [startDate[0] + 6]
for index, row in islice(df_checkCumRpt.iterrows(), 1, None):
    startDate.append(startDate[index-1]+7)
    endDate.append(endDate[index-1]+7)
df_checkCumRpt['week start'] = (startDate)
df_checkCumRpt['end'] = (endDate)
df_checkCumRpt

,week,Cum. Rpt. Dons.,Weekly Rpt. Dons.,week start,end
0,1,0.009179,0.009179,40544.0,40550.0
1,2,0.039975,0.030796,40551.0,40557.0
2,3,0.096740,0.056765,40558.0,40564.0
3,4,0.202092,0.105352,40565.0,40571.0
4,5,0.360013,0.157921,40572.0,40578.0
...,...,...,...,...,...
391,392,3422.249158,7.223329,43281.0,43287.0
392,393,3429.466770,7.217611,43288.0,43294.0
393,394,3436.678682,7.211913,43295.0,43301.0
394,395,3443.884915,7.206232,43302.0,43308.0


In [59]:
t = [0]
t2 = [6/365]
for index, row in islice(df_checkCumRpt.iterrows(), 1, None):
    t.append(t[index-1]+7/365)
    t2.append(t2[index-1]+7/365)
df_checkCumRpt["num rpt don"] = ""
df_checkCumRpt["cumul."] = ""
df_checkCumRpt['t'] = (t)
df_checkCumRpt['t2'] = (t2)
df_checkCumRpt

,week,Cum. Rpt. Dons.,Weekly Rpt. Dons.,week start,end,num rpt don,cumul.,t,t2
0,1,0.009179,0.009179,40544.0,40550.0,,,0.000000,0.016438
1,2,0.039975,0.030796,40551.0,40557.0,,,0.019178,0.035616
2,3,0.096740,0.056765,40558.0,40564.0,,,0.038356,0.054795
3,4,0.202092,0.105352,40565.0,40571.0,,,0.057534,0.073973
4,5,0.360013,0.157921,40572.0,40578.0,,,0.076712,0.093151
...,...,...,...,...,...,...,...,...,...
391,392,3422.249158,7.223329,43281.0,43287.0,,,7.498630,7.515068
392,393,3429.466770,7.217611,43288.0,43294.0,,,7.517808,7.534247
393,394,3436.678682,7.211913,43295.0,43301.0,,,7.536986,7.553425
394,395,3443.884915,7.206232,43302.0,43308.0,,,7.556164,7.572603


In [60]:
numRpt = []
cummul = []
weekStart = df_checkCumRpt['week start'].tolist()
weekEnd = df_checkCumRpt['end'].tolist()
t = df_checkCumRpt['t'].tolist()
t2 = df_checkCumRpt['t2'].tolist()
count99 = 0
countTotal99 = 0
for index, row in islice(df_checkCumRpt.iterrows(), 0, None):
    count99 = count99 + len(dfraw2.loc[(dfraw2['Gift Date']>=weekStart[index]) & (dfraw2['Gift Date']<=weekEnd[index])])
    count99 = count99 - len(df_ns.loc[df_ns['time of 1st purchase']>=(t[index]-.001)])
    count99 = count99 + len(df_ns.loc[df_ns['time of 1st purchase']>(t2[index]+.001)])
    countTotal99 = countTotal99+count99
    numRpt.append(count99)
    cummul.append(countTotal99)
    count99 = 0
df_checkCumRpt["num rpt don"] = numRpt
df_checkCumRpt["cumul."] = cummul
df_checkCumRpt

,week,Cum. Rpt. Dons.,Weekly Rpt. Dons.,week start,end,num rpt don,cumul.,t,t2
0,1,0.009179,0.009179,40544.0,40550.0,3,3,0.000000,0.016438
1,2,0.039975,0.030796,40551.0,40557.0,3,6,0.019178,0.035616
2,3,0.096740,0.056765,40558.0,40564.0,0,6,0.038356,0.054795
3,4,0.202092,0.105352,40565.0,40571.0,0,6,0.057534,0.073973
4,5,0.360013,0.157921,40572.0,40578.0,3,9,0.076712,0.093151
...,...,...,...,...,...,...,...,...,...
391,392,3422.249158,7.223329,43281.0,43287.0,6,3539,7.498630,7.515068
392,393,3429.466770,7.217611,43288.0,43294.0,7,3546,7.517808,7.534247
393,394,3436.678682,7.211913,43295.0,43301.0,6,3552,7.536986,7.553425
394,395,3443.884915,7.206232,43302.0,43308.0,0,3552,7.556164,7.572603


In [61]:
df_allCondExp = df_BGNBD.iloc[:,0:5].copy(deep=True)
endDate = beginning_date + .5*(predict_thru-beginning_date)
constT = (predict_thru - endDate)/365
df_allCondExp['t'] = constT
df_allCondExp["E(Y(t)|X=x,t_x,T)"] = ""
df_allCondExp["2F1"] = 0
df_allCondExp["a"] = df_allCondExp.iloc[:,2]+r
df_allCondExp["b"] = df_allCondExp.iloc[:,2] + b
df_allCondExp["c"] = df_allCondExp.iloc[:,2] + b + a - 1
df_allCondExp["z"] = constT/(alpha + df_allCondExp.iloc[:,4] + constT)
df_allCondExp['0'] = 1
df_allCondExp

,Constituent ID,Name,x (#donations),t_x (last gift),T (total time span),t,"E(Y(t)|X=x,t_x,T)",2F1,a,b,c,z,0
0,1927780,"Angeles, Joey D.",25,3.208219,3.794521,3.8,,0,25.147642,26.366944,25.694801,0.478222,1
1,1927778,"Ayika, Christopher",0,0.000000,3.791781,3.8,,0,0.147642,1.366944,0.694801,0.478387,1
2,1927784,"Boland, Patrick T.",6,0.172603,3.786301,3.8,,0,6.147642,7.366944,6.694801,0.478717,1
3,1927785,"Callahan, Patrick J.",0,0.000000,3.775342,3.8,,0,0.147642,1.366944,0.694801,0.479379,1
4,1927788,"Rowell, Esperanza",3,3.021918,3.767123,3.8,,0,3.147642,4.366944,3.694801,0.479877,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021,1930534,"Chmielarczyk, Allison",0,0.000000,2.638356,3.8,,0,0.147642,1.366944,0.694801,0.559652,1
2022,1929858,"Grove, Karen E.",3,2.350685,2.635616,3.8,,0,3.147642,4.366944,3.694801,0.559878,1
2023,1929859,"Leipr, Michael H.",0,0.000000,2.635616,3.8,,0,0.147642,1.366944,0.694801,0.559878,1
2024,1929860,"Foy, Charles B.",0,0.000000,2.635616,3.8,,0,0.147642,1.366944,0.694801,0.559878,1


In [62]:
aceCount = 1
aceArr = []
while (aceCount <= 200):
    df_allCondExp[str(aceCount)]=df_allCondExp.iloc[:,aceCount+11]*(df_allCondExp.iloc[:,8]+aceCount-1)*(df_allCondExp.iloc[:,9]+aceCount-1)/((df_allCondExp.iloc[:,10]+aceCount-1)*aceCount)*df_allCondExp.iloc[:,11]
    aceCount = aceCount + 1
df_allCondExp

,Constituent ID,Name,x (#donations),t_x (last gift),T (total time span),t,"E(Y(t)|X=x,t_x,T)",2F1,a,b,...,191,192,193,194,195,196,197,198,199,200
0,1927780,"Angeles, Joey D.",25,3.208219,3.794521,3.8,,0,25.147642,26.366944,...,1.511397e-29,8.162111e-30,4.405234e-30,2.376185e-30,1.280970e-30,6.901561e-31,3.716276e-31,1.999968e-31,1.075710e-31,5.782641e-32
1,1927778,"Ayika, Christopher",0,0.000000,3.791781,3.8,,0,0.147642,1.366944,...,6.213219e-63,2.969505e-63,1.419233e-63,6.783054e-64,3.241896e-64,1.549441e-64,7.405475e-65,3.539427e-65,1.691667e-65,8.085353e-66
2,1927784,"Boland, Patrick T.",6,0.172603,3.786301,3.8,,0,6.147642,7.366944,...,3.008439e-51,1.483832e-51,7.317493e-52,3.608065e-52,1.778777e-52,8.768080e-53,4.321395e-53,2.129514e-53,1.049240e-53,5.169010e-54
3,1927785,"Callahan, Patrick J.",0,0.000000,3.775342,3.8,,0,0.147642,1.366944,...,9.229022e-63,4.420005e-63,2.116859e-63,1.013826e-63,4.855532e-64,2.325479e-64,1.113756e-64,5.334205e-65,2.554768e-65,1.223588e-65
4,1927788,"Rowell, Esperanza",3,3.021918,3.767123,3.8,,0,3.147642,4.366944,...,6.456413e-56,3.143755e-56,1.530642e-56,7.451886e-57,3.627664e-57,1.765860e-57,8.595167e-58,4.183324e-58,2.035907e-58,9.907503e-59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021,1930534,"Chmielarczyk, Allison",0,0.000000,2.638356,3.8,,0,0.147642,1.366944,...,6.423878e-50,3.591727e-50,2.008221e-50,1.122850e-50,6.278187e-51,3.510337e-51,1.962752e-51,1.097448e-51,6.136274e-52,3.431053e-52
2022,1929858,"Grove, Karen E.",3,2.350685,2.635616,3.8,,0,3.147642,4.366944,...,3.981592e-43,2.261923e-43,1.284891e-43,7.298318e-44,4.145217e-44,2.354183e-44,1.336909e-44,7.591590e-45,4.310553e-45,2.447388e-45
2023,1929859,"Leipr, Michael H.",0,0.000000,2.635616,3.8,,0,0.147642,1.366944,...,6.938639e-50,3.881107e-50,2.170896e-50,1.214296e-50,6.792230e-51,3.799288e-51,2.125172e-51,1.188743e-51,6.649423e-52,3.719477e-52
2024,1929860,"Foy, Charles B.",0,0.000000,2.635616,3.8,,0,0.147642,1.366944,...,6.938639e-50,3.881107e-50,2.170896e-50,1.214296e-50,6.792230e-51,3.799288e-51,2.125172e-51,1.188743e-51,6.649423e-52,3.719477e-52


In [63]:
aceCount1 = 0
while (aceCount1 <= 200):
    df_allCondExp["2F1"] = df_allCondExp["2F1"]+df_allCondExp.iloc[:,aceCount1+12]
    aceCount1 = aceCount1 + 1
df_allCondExp

,Constituent ID,Name,x (#donations),t_x (last gift),T (total time span),t,"E(Y(t)|X=x,t_x,T)",2F1,a,b,...,191,192,193,194,195,196,197,198,199,200
0,1927780,"Angeles, Joey D.",25,3.208219,3.794521,3.8,,1.956437e+07,25.147642,26.366944,...,1.511397e-29,8.162111e-30,4.405234e-30,2.376185e-30,1.280970e-30,6.901561e-31,3.716276e-31,1.999968e-31,1.075710e-31,5.782641e-32
1,1927778,"Ayika, Christopher",0,0.000000,3.791781,3.8,,1.233707e+00,0.147642,1.366944,...,6.213219e-63,2.969505e-63,1.419233e-63,6.783054e-64,3.241896e-64,1.549441e-64,7.405475e-65,3.539427e-65,1.691667e-65,8.085353e-66
2,1927784,"Boland, Patrick T.",6,0.172603,3.786301,3.8,,8.274242e+01,6.147642,7.366944,...,3.008439e-51,1.483832e-51,7.317493e-52,3.608065e-52,1.778777e-52,8.768080e-53,4.321395e-53,2.129514e-53,1.049240e-53,5.169010e-54
3,1927785,"Callahan, Patrick J.",0,0.000000,3.775342,3.8,,1.234547e+00,0.147642,1.366944,...,9.229022e-63,4.420005e-63,2.116859e-63,1.013826e-63,4.855532e-64,2.325479e-64,1.113756e-64,5.334205e-65,2.554768e-65,1.223588e-65
4,1927788,"Rowell, Esperanza",3,3.021918,3.767123,3.8,,1.154858e+01,3.147642,4.366944,...,6.456413e-56,3.143755e-56,1.530642e-56,7.451886e-57,3.627664e-57,1.765860e-57,8.595167e-58,4.183324e-58,2.035907e-58,9.907503e-59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021,1930534,"Chmielarczyk, Allison",0,0.000000,2.638356,3.8,,1.312902e+00,0.147642,1.366944,...,6.423878e-50,3.591727e-50,2.008221e-50,1.122850e-50,6.278187e-51,3.510337e-51,1.962752e-51,1.097448e-51,6.136274e-52,3.431053e-52
2022,1929858,"Grove, Karen E.",3,2.350685,2.635616,3.8,,2.166158e+01,3.147642,4.366944,...,3.981592e-43,2.261923e-43,1.284891e-43,7.298318e-44,4.145217e-44,2.354183e-44,1.336909e-44,7.591590e-45,4.310553e-45,2.447388e-45
2023,1929859,"Leipr, Michael H.",0,0.000000,2.635616,3.8,,1.313157e+00,0.147642,1.366944,...,6.938639e-50,3.881107e-50,2.170896e-50,1.214296e-50,6.792230e-51,3.799288e-51,2.125172e-51,1.188743e-51,6.649423e-52,3.719477e-52
2024,1929860,"Foy, Charles B.",0,0.000000,2.635616,3.8,,1.313157e+00,0.147642,1.366944,...,6.938639e-50,3.881107e-50,2.170896e-50,1.214296e-50,6.792230e-51,3.799288e-51,2.125172e-51,1.188743e-51,6.649423e-52,3.719477e-52


In [64]:
longNameArr = []
#((a+b+df_allCondExp.iloc[:,2]-1)/(a-1)*(1-pow(((alpha+df_allCondExp.iloc[:,4])/(alpha+df_allCondExp.iloc[:,4]+df_allCondExp.iloc[:,5])),(r+df_allCondExp.iloc[:,2]))*df_allCondExp.iloc[:,7])/(1+(df_allCondExp.iloc[:,2]>0)*a/(b+df_allCondExp.iloc[:,2]-1)*pow(((alpha+df_allCondExp.iloc[:,4])/(alpha+df_allCondExp.iloc[:,3])),(r+df_allCondExp.iloc[:,2]))))
for index, row in islice(df_allCondExp.iterrows(), 0, None):
    longNameArr.append((a+b+df_allCondExp.iloc[index,2]-1)/(a-1)*(1-pow(((alpha+df_allCondExp.iloc[index,4])/(alpha+df_allCondExp.iloc[index,4]+df_allCondExp.iloc[index,5])),(r+df_allCondExp.iloc[index,2]))*df_allCondExp.iloc[index,7])/(1+(df_allCondExp.iloc[index,2]>0)*a/(b+df_allCondExp.iloc[index,2]-1)*pow(((alpha+df_allCondExp.iloc[index,4])/(alpha+df_allCondExp.iloc[index,3])),(r+df_allCondExp.iloc[index,2]))))
df_allCondExp["E(Y(t)|X=x,t_x,T)"] = longNameArr
df_allCondExp

,Constituent ID,Name,x (#donations),t_x (last gift),T (total time span),t,"E(Y(t)|X=x,t_x,T)",2F1,a,b,...,191,192,193,194,195,196,197,198,199,200
0,1927780,"Angeles, Joey D.",25,3.208219,3.794521,3.8,12.866621,1.956437e+07,25.147642,26.366944,...,1.511397e-29,8.162111e-30,4.405234e-30,2.376185e-30,1.280970e-30,6.901561e-31,3.716276e-31,1.999968e-31,1.075710e-31,5.782641e-32
1,1927778,"Ayika, Christopher",0,0.000000,3.791781,3.8,0.124746,1.233707e+00,0.147642,1.366944,...,6.213219e-63,2.969505e-63,1.419233e-63,6.783054e-64,3.241896e-64,1.549441e-64,7.405475e-65,3.539427e-65,1.691667e-65,8.085353e-66
2,1927784,"Boland, Patrick T.",6,0.172603,3.786301,3.8,0.000299,8.274242e+01,6.147642,7.366944,...,3.008439e-51,1.483832e-51,7.317493e-52,3.608065e-52,1.778777e-52,8.768080e-53,4.321395e-53,2.129514e-53,1.049240e-53,5.169010e-54
3,1927785,"Callahan, Patrick J.",0,0.000000,3.775342,3.8,0.125209,1.234547e+00,0.147642,1.366944,...,9.229022e-63,4.420005e-63,2.116859e-63,1.013826e-63,4.855532e-64,2.325479e-64,1.113756e-64,5.334205e-65,2.554768e-65,1.223588e-65
4,1927788,"Rowell, Esperanza",3,3.021918,3.767123,3.8,2.210314,1.154858e+01,3.147642,4.366944,...,6.456413e-56,3.143755e-56,1.530642e-56,7.451886e-57,3.627664e-57,1.765860e-57,8.595167e-58,4.183324e-58,2.035907e-58,9.907503e-59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021,1930534,"Chmielarczyk, Allison",0,0.000000,2.638356,3.8,0.168666,1.312902e+00,0.147642,1.366944,...,6.423878e-50,3.591727e-50,2.008221e-50,1.122850e-50,6.278187e-51,3.510337e-51,1.962752e-51,1.097448e-51,6.136274e-52,3.431053e-52
2022,1929858,"Grove, Karen E.",3,2.350685,2.635616,3.8,3.084396,2.166158e+01,3.147642,4.366944,...,3.981592e-43,2.261923e-43,1.284891e-43,7.298318e-44,4.145217e-44,2.354183e-44,1.336909e-44,7.591590e-45,4.310553e-45,2.447388e-45
2023,1929859,"Leipr, Michael H.",0,0.000000,2.635616,3.8,0.168808,1.313157e+00,0.147642,1.366944,...,6.938639e-50,3.881107e-50,2.170896e-50,1.214296e-50,6.792230e-51,3.799288e-51,2.125172e-51,1.188743e-51,6.649423e-52,3.719477e-52
2024,1929860,"Foy, Charles B.",0,0.000000,2.635616,3.8,0.168808,1.313157e+00,0.147642,1.366944,...,6.938639e-50,3.881107e-50,2.170896e-50,1.214296e-50,6.792230e-51,3.799288e-51,2.125172e-51,1.188743e-51,6.649423e-52,3.719477e-52


In [65]:
df_pAlive = df_BGNBD.iloc[:,0:5].copy(deep=True)
paliveinfo = []
for index, row in islice(df_allCondExp.iterrows(), 0, None):
    paliveinfo.append(1/(1+(df_pAlive.iloc[index,2]>0)*(a/(b+df_pAlive.iloc[index,2]-1))*(pow(((alpha+df_pAlive.iloc[index,4])/(alpha+df_pAlive.iloc[index,3])),(r+df_pAlive.iloc[index,2])))))
df_pAlive["P(Alive) Info"] = paliveinfo
df_pAlive

,Constituent ID,Name,x (#donations),t_x (last gift),T (total time span),P(Alive) Info
0,1927780,"Angeles, Joey D.",25,3.208219,3.794521,0.625862
1,1927778,"Ayika, Christopher",0,0.000000,3.791781,1.000000
2,1927784,"Boland, Patrick T.",6,0.172603,3.786301,0.000059
3,1927785,"Callahan, Patrick J.",0,0.000000,3.775342,1.000000
4,1927788,"Rowell, Esperanza",3,3.021918,3.767123,0.845659
...,...,...,...,...,...,...
2021,1930534,"Chmielarczyk, Allison",0,0.000000,2.638356,1.000000
2022,1929858,"Grove, Karen E.",3,2.350685,2.635616,0.882222
2023,1929859,"Leipr, Michael H.",0,0.000000,2.635616,1.000000
2024,1929860,"Foy, Charles B.",0,0.000000,2.635616,1.000000


In [66]:
actNonlap = []
df_pAlive["Active"] = ~dfdp2["Inactive?"]*1
for index, row in islice(df_pAlive.iterrows(), 0, None):
    if df_pAlive.iloc[index,6] == 0:
        actNonlap.append(0)
    else:
        if (df_pAlive.iloc[index,4]-df_pAlive.iloc[index,3])>2:
            actNonlap.append(0)
        else:
            actNonlap.append(1)
df_pAlive["Act/Nonlap"] = actNonlap
df_pAlive

,Constituent ID,Name,x (#donations),t_x (last gift),T (total time span),P(Alive) Info,Active,Act/Nonlap
0,1927780,"Angeles, Joey D.",25,3.208219,3.794521,0.625862,1,1
1,1927778,"Ayika, Christopher",0,0.000000,3.791781,1.000000,0,0
2,1927784,"Boland, Patrick T.",6,0.172603,3.786301,0.000059,0,0
3,1927785,"Callahan, Patrick J.",0,0.000000,3.775342,1.000000,0,0
4,1927788,"Rowell, Esperanza",3,3.021918,3.767123,0.845659,1,1
...,...,...,...,...,...,...,...,...
2021,1930534,"Chmielarczyk, Allison",0,0.000000,2.638356,1.000000,0,0
2022,1929858,"Grove, Karen E.",3,2.350685,2.635616,0.882222,1,1
2023,1929859,"Leipr, Michael H.",0,0.000000,2.635616,1.000000,0,0
2024,1929860,"Foy, Charles B.",0,0.000000,2.635616,1.000000,0,0


In [67]:
df_onlyx = df_pAlive.copy(deep=True)
df_onlyx

,Constituent ID,Name,x (#donations),t_x (last gift),T (total time span),P(Alive) Info,Active,Act/Nonlap
0,1927780,"Angeles, Joey D.",25,3.208219,3.794521,0.625862,1,1
1,1927778,"Ayika, Christopher",0,0.000000,3.791781,1.000000,0,0
2,1927784,"Boland, Patrick T.",6,0.172603,3.786301,0.000059,0,0
3,1927785,"Callahan, Patrick J.",0,0.000000,3.775342,1.000000,0,0
4,1927788,"Rowell, Esperanza",3,3.021918,3.767123,0.845659,1,1
...,...,...,...,...,...,...,...,...
2021,1930534,"Chmielarczyk, Allison",0,0.000000,2.638356,1.000000,0,0
2022,1929858,"Grove, Karen E.",3,2.350685,2.635616,0.882222,1,1
2023,1929859,"Leipr, Michael H.",0,0.000000,2.635616,1.000000,0,0
2024,1929860,"Foy, Charles B.",0,0.000000,2.635616,1.000000,0,0


In [68]:
dfraw2.to_excel(writer, 'Raw data2', index = False)
dfraw1.to_excel(writer, 'Raw data1', index = False)
dfraw1.to_excel(writer, 'Data prep1', index = False)
dfdp2.to_excel(writer, 'Data prep2', index = False)
dfmd.to_excel(writer, 'Model data', index = False)
df_BGNBD.to_excel(writer, "BGNBD Estimation", index = False)
df_ext.to_excel(writer, "E(X(t))", index = False)
df_ns.to_excel(writer, "n_s", index=False)
df_sls.to_excel(writer, "CumRptSls", index=False)
df_checkCumRpt.to_excel(writer, "Check CumRpt", index=False)
df_allCondExp.to_excel(writer, "All Cond. Exp.", index=False)
df_pAlive.to_excel(writer, "P(alive)", index=False)
df_onlyx.to_excel(writer, "only x>0", index=False)
writer.save()